Import Pandas package so we can use Pandas dataframes

In [12]:
import pandas as pd

Function to load the .csv files containing the static tables. This function takes a list as input where each list item is the string of a pathway to a .csv file. The files are then opened as Pandas dataframes and stored in a dictionary where the filename is the key and the dataframe is the value.

In [13]:
def Load_csvs(list):
    dic_of_dfs = {}
    for i in list:
        dfname = i.rsplit('\\', 1)[-1]
        dic_of_dfs[dfname] = pd.read_csv(i)
    return dic_of_dfs